In [1]:
import fitz  # PyMuPDF
from easyocr import Reader
import numpy as np
import os
import cv2
import pickle  
import torch

In [2]:

reader = Reader(['pt'], gpu=True, verbose=False, download_enabled=True, recognizer=True)  # Altere conforme o idioma do mangá

def preprocess_image(img_array):
    """Aplica pré-processamento para melhorar a detecção de texto"""
    # Conversão para escala de cinza
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    
    # Redução de ruído com filtro bilateral (preserva bordas)
    denoised = cv2.bilateralFilter(gray, 9, 75, 75)
    
    # Equalização de histograma adaptativo para melhorar contraste
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced = clahe.apply(denoised)
    
    # Binarização adaptativa para lidar com variações de iluminação
    thresh = cv2.adaptiveThreshold(contrast_enhanced, 255, 
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, 11, 2)
    
    # Operação morfológica para fechar pequenas lacunas no texto
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    processed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    # Inversão de cores se necessário (para texto claro em fundo escuro)
    if np.mean(processed) < 127:
        processed = cv2.bitwise_not(processed)
    
    return cv2.cvtColor(processed, cv2.COLOR_GRAY2RGB)

def process_page(page):
    """Processa uma única página com pré-processamento e EasyOCR"""
    pix = page.get_pixmap()
    img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
    
    # Aplica pré-processamento
    img_processed = preprocess_image(img_array)
    
    # Detecção de texto com ajustes para mangás
    results = reader.readtext(img_processed,
                             paragraph=True,
                             text_threshold=0.4,
                             link_threshold=0.4,
                             rotation_info=[90, 180, 270])
    
    return " ".join([result[1] for result in results])

In [3]:

def extract_text_with_easyocr(pdf_path, numberFile):
    """Processa PDF em batches para otimizar memória."""
    doc = fitz.open(pdf_path)
    text = []
    pages = [doc.load_page(i) for i in range(len(doc))]
    
    for i in range(0, len(pages)):
        batch_text = process_page(pages[i])
        text.extend(batch_text)
    with open(os.path.join("C:/Repositorios/RAG-Anime/onePiecePickles", f"{numberFile}.pkl"), "wb") as f:
        pickle.dump(text, f)       
    torch.cuda.empty_cache()

In [4]:
print(torch.cuda.memory_summary())
torch.cuda.empty_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 196503 KiB | 196503 KiB | 212951 KiB |  16448 KiB |
|       from large pool | 184592 KiB | 184592 KiB | 192816 KiB |   8224 KiB |
|       from small pool |  11911 KiB |  11911 KiB |  20135 KiB |   8224 KiB |
|---------------------------------------------------------------------------|
| Active memory         | 196503 KiB | 196503 KiB | 212951 KiB |  16448 KiB |
|       from large pool | 184592 KiB | 184592 KiB | 192816 KiB |

In [5]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 196503 KiB | 196503 KiB | 212951 KiB |  16448 KiB |
|       from large pool | 184592 KiB | 184592 KiB | 192816 KiB |   8224 KiB |
|       from small pool |  11911 KiB |  11911 KiB |  20135 KiB |   8224 KiB |
|---------------------------------------------------------------------------|
| Active memory         | 196503 KiB | 196503 KiB | 212951 KiB |  16448 KiB |
|       from large pool | 184592 KiB | 184592 KiB | 192816 KiB |

In [6]:

files = os.listdir('E:/One Piece Manga/')
for folder in files:
    if not folder.endswith('.pdf'):
        filesPDF = os.listdir(f'E:/One Piece Manga/{folder}/{folder}/')
        for pdf in filesPDF:
            extract_text_with_easyocr(f'E:/One Piece Manga/{folder}/{folder}/' + pdf, pdf.split(".")[1])
            print(f"Finalizado {pdf}")

Finalizado One Piece Vol.01.pdf
Finalizado One Piece Vol.02.pdf
Finalizado One Piece Vol.03.pdf
Finalizado One Piece Vol.04.pdf
Finalizado One Piece Vol.05.pdf
Finalizado One Piece Vol.06.pdf
Finalizado One Piece Vol.07.pdf
Finalizado One Piece Vol.08.pdf
Finalizado One Piece Vol.09.pdf
Finalizado One Piece Vol.10.pdf
Finalizado One Piece Vol.101.pdf
Finalizado One Piece Vol.102.pdf
Finalizado One Piece Vol.103.pdf
Finalizado One Piece Vol.104.pdf
Finalizado One Piece Vol.105.pdf
Finalizado One Piece Vol.106.pdf
Finalizado One Piece Vol.107.pdf
Finalizado One Piece Vol.108.pdf
Finalizado One Piece Vol.109.pdf
Finalizado One Piece Vol.110.pdf
Finalizado One Piece Vol.11.pdf
Finalizado One Piece Vol.12.pdf
Finalizado One Piece Vol.13.pdf
Finalizado One Piece Vol.14.pdf
Finalizado One Piece Vol.15.pdf
Finalizado One Piece Vol.16.pdf
Finalizado One Piece Vol.17.pdf
Finalizado One Piece Vol.18.pdf
Finalizado One Piece Vol.19.pdf
Finalizado One Piece Vol.20.pdf
Finalizado One Piece Vol.111.p

In [6]:


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings  # Altere conforme necessário

# 1. Configuração do Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

# 2. Configuração do Modelo de Embedding (exemplo com HuggingFace)
embedding_model = HuggingFaceEmbeddings(
    model_name="neuralmind/bert-base-portuguese-cased",  # Altere para o modelo desejado
    model_kwargs={'device': 'cuda'},  # ou 'cpu' se não tiver GPU
    encode_kwargs={'normalize_embeddings': False}
)


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


In [11]:
def process_volumes(pickle_dir, output_dir="embeddingsOnePiece"):
    os.makedirs(output_dir, exist_ok=True)
    
    all_chunks = []
    
    # Processar cada arquivo pickle
    for filename in os.listdir(pickle_dir):
        if filename.endswith(".pkl"):
            volume_path = os.path.join(pickle_dir, filename)
            
            # Carregar o texto
            with open(volume_path, "rb") as f:
                volume_text = pickle.load(f)
            
            # Converter lista para string se necessário
            if isinstance(volume_text, list):
                volume_text = " ".join(volume_text)  # Concatena todos os elementos da lista
                
            # Aplicar split de texto
            chunks = text_splitter.split_text(volume_text)
            
            # Gerar embeddings para cada chunk
            volume_embeddings = embedding_model.embed_documents(chunks)
            
            # Salvar resultados
            output_data = {
                'volume_name': filename,
                'chunks': chunks,
                'embeddings': volume_embeddings
            }
            
            output_path = os.path.join(output_dir, f"embeddings_{filename}")
            with open(output_path, "wb") as f:
                pickle.dump(output_data, f)
            
            all_chunks.extend(zip(chunks, volume_embeddings))
    
    return all_chunks

# 4. Executar o processamento
volumes_dir = "C:/Repositorios/RAG-Anime/onePiecePickles"  # Altere para seu diretório
resultado_embeddings = process_volumes(volumes_dir)

In [13]:
from langchain.vectorstores import FAISS  # Exemplo com FAISS

# Criar vetorstore
vectorstore = FAISS.from_texts(
    texts=[chunk[0] for chunk in resultado_embeddings],
    embedding=embedding_model,
    metadatas=[{"volume": chunk[1]} for chunk in resultado_embeddings]
)

# Salvar para uso futuro
vectorstore.save_local("one_piece_faiss_index")